<a href="https://colab.research.google.com/github/sakuronohana/cas_datenanalyse/blob/master/Semesterarbeit/CAS_DA_SA_Payment_Fraud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img 
src="https://www.ffhs.ch/templates/ffhs/img/logo@2x.png" width="100"> 
###DaAn, Data Analysis, MAS/CAS Web4B 2018, ZH1, FS19, Dr. Tödtli Beat###

##*Semesterarbeit von Patrik Di Lena*##
#Betrugserkennung Zahlungsverkehr#



##Ziel##
Auf der Grundlage von maschinellen Lernalgorithmen soll ein Betrugserkennungssystem für den Zahlungsverkehr einer ausschliesslich in der Schweiz tätigen Bank erstellt werden. Auf Basis historisierter Daten in Bezug auf das E-Banking Session- und Zahlungsverhalten von Bankkunden sollen mögliche Betrugsintensionen erkannt und für bestimmte Risikogruppen vorhergesagt werden.

##Aufgabenstellung##

Aus dem oben formulierten Ziel ergeben sich zwei wesentliche Aufgabenstellungen:
1.	Erkennung betrügerischer Handlungen im Zahlungsverkehr
2.	Ermittlung potenzieller Kundenrisikogruppen

##Betrugsanfällige Risikogruppen##
Die für den Betrug im Zahlungsverkehr anfälligen Gruppen werden auf Basis der folgenden Merkmale durch den Lernalgorithmus bestimmt:

*	Geschlecht
*	Alter
*	Zivilstand
*	Nationalität
*	Kontosaldo
*	Wohnort
*	Region
*	Zahlungsmethode (E-Rechnung, Einzahlungsschein etc.)
*	Rechtsform (Juristisch oder natürliche Person)

## Datensatz##
Die im Rahmen dieser Semesterarbeit verwendeten Daten entstammen, von ihrer Datenstruktur her, aus einem bereits eingesetzten nicht ML-basierten E-Banking Betrugserkennungssystem. Die Datenwerte wurde zu Wahrung der datenschutzrechtlichen und bankengesetzlichen Aspekte mit Hilfe des folgenden Python-Scripts künstlich erzeugt und beinhalten somit keine reale Zahlungs-, Kunden-, und betrugsspezifischen Informationen. Weiter wurden einige durch die Bank definierten Spezifikationen bzw. Merkmale aus sicherheitstechnischen Gründen umbenannt.  

Datenselektion:

- 5'000 synthetische Kunden (Vorname, Name, Strasse, Ort, Kanton, Sprachregion, Geschlecht, Alter, Zivilstand,  Nationalität, Kontonummer, Vertragsnummer, Rechtsform )

- 100'000 synthetische Zahlungen (Transaktionsid, ,Erstellungdatum/Zeit, Empfängerkonto, Empfängerbank, Empfängerland, Währung, Betrag, Valuta Datum)








###Datengeneratoren###

Während die Stammdaten hauptsächlich im Excel manuell von verschiedenen Quellen 

Datenquellen:

Open Government Data https://opendata.swiss/de/


###IBAN Generator###

In [107]:
# IBAN Generator

import datetime
import numpy as np
import pandas as pd

# Datenquellen Github
ibandb_url = 'https://raw.githubusercontent.com/sakuronohana/cas_datenanalyse/master/Semesterarbeit/Dataset/iban_list.csv'

# IBAN Generator in elektronischer Form

iban_cc = pd.read_csv(ibandb_url, delimiter=';')
iban_cc.columns=['Country','Code','Checkdig','Length','Sample','Bank']

# IBAN Gen. 1.Teil - Erstellung IBAN Länder- und Bankencode
code = np.random.choice(np.ravel(iban_cc['Code']).astype(str))
checkdig = np.random.choice(np.ravel(iban_cc['Checkdig']).astype(str))
iban_country_code = code + checkdig
iban_bank_code = np.random.choice((iban_cc.loc[iban_cc['Code'] == code])['Length']).astype(str)
iban_ccbc = iban_country_code + "00" + iban_bank_code 

# IBAN Gen. 2.Teil - Erstellung Kontonummer und Zusammensetzung der IBAN
eiban = []
for row in range(20):
    part_eiban = pd.DataFrame(np.int64(np.random.randint(2000000000000,3000000000000,20800)))
    eiban = iban_ccbc + part_eiban.astype(str)

# Zufällige Auswahl einer IBAN aus dem generierten Datensatz
eiban = np.ravel(eiban)
eiban = np.random.choice(eiban)
print (eiban)





LC7200322800994985921


###Payment Generator###

In [11]:
# Erstellung von Transaktionsdaten 
# Daten Payment ID z.B. 1059295797

from datetime import datetime, timedelta
import random
import numpy as np
import pandas as pd

# Datenquellen Github
ibandb_url = 'https://raw.githubusercontent.com/sakuronohana/cas_datenanalyse/master/Semesterarbeit/Dataset/iban_list.csv'

# IBAN Generator in elektronischer Form

iban_cc = pd.read_csv(ibandb_url, delimiter=';')
iban_cc.columns=['Country','Code','Checkdig','Length','IBAN','Bank']

# IBAN Gen. 1.Teil - Erstellung IBAN Länder- und Bankencode
code = np.random.choice(np.ravel(iban_cc['Code']).astype(str))
checkdig = np.random.choice(np.ravel(iban_cc['Checkdig']).astype(str))
iban_country_code = code + checkdig
iban_bank_code = np.random.choice((iban_cc.loc[iban_cc['Code'] == code])['Length']).astype(str)
iban_ccbc = iban_country_code + "00" + iban_bank_code 


# IBAN Gen. 2.Teil - Erstellung Kontonummer und Zusammensetzung der IBAN
eiban = []
for row in range(20):
    part_eiban = pd.DataFrame(np.int64(np.random.randint(2000000000000,3000000000000,20800)))
    eiban = iban_ccbc + part_eiban.astype(str)

# Zufällige Auswahl einer IBAN aus dem generierten Datensatz
eiban = np.ravel(eiban)
eiban = np.random.choice(eiban)


# Datenquellen Github
custdb_url = 'https://raw.githubusercontent.com/sakuronohana/cas_datenanalyse/master/Semesterarbeit/Dataset/cutomer_base_data.csv'

cust_data = pd.read_csv(custdb_url, delimiter=';')
cust_data.columns=['anrede','vorname','nachname','strasse','nr','plz','ort',
                   'kanton','sprachregion',	'nation',	'sexcode','mobile_nr',	
                   'vertrag_nr','konto_nr','auth_device']

# Importierte Kundendaten bereinigen
cust_data = cust_data.fillna(0)
cust_data_dtype = {'plz':int,'mobile_nr':int,'vertrag_nr':int}
cust_data = cust_data.astype(cust_data_dtype)

# Zufallsauswahl Kundendaten
cd_contract = np.random.choice(np.ravel(cust_data['vertrag_nr']))
cd_kontonr = cust_data.loc[cust_data['vertrag_nr'] == cd_contract,['konto_nr']]
#get_cids = cust_data.loc[cust_data['vertrag_nr'] == cd_contract,['konto_nr','vertrag_nr',
                                                                 #'nachname','vorname','sexcode',
                                                                 #'plz','ort','kanton',
                                                                #'sprachregion','auth_device']]





# Erstellung der Zahlungs-Basisinformationen 

#start_date = datetime(2018,1,1,8,0,0)
trx_data = pd.DataFrame(columns=['timestamp','paym_id','send_bc_nr','amount',
                                 'rcv_bc_nr','rcv_bc_iban','rcv_bc_country','rcv_bc_code',
                                 'valuta_date','cust_vertrag_nr','cust_konto_nr',
                                 'cust_vname','cust_nname','cust_plz','cust_ort',
                                'cust_kanton','cust_sprachregion','cust_auth_device'])

for i in range(30):
    trx_data.loc[i,['timestamp']] = timestamps = pd.Timestamp(datetime(2018,1,1,8,0,0)+timedelta(seconds=30*i)).strftime("%Y.%m.%d %H:%M")
    trx_data.loc[i,['paym_id']] =  np.int64(np.random.randint(1000000000000,2000000000000,1))
    trx_data.loc[i,['send_bc_nr']] =  'CHBANK71XXX'
    trx_data.loc[i,['amount']] = np.int64(np.random.randint(100,100000,1)).astype(str)
    trx_data.loc[i,['rcv_bc_nr']] = bc_bank = np.random.choice(np.ravel(iban_cc['Bank']).astype(str))
    trx_data.loc[i,['rcv_bc_iban']] = np.ravel(iban_cc.loc[iban_cc['Bank'] == bc_bank,['IBAN']])
    trx_data.loc[i,['rcv_bc_country']] = np.ravel(iban_cc.loc[iban_cc['Bank'] == bc_bank,['Country']])
    trx_data.loc[i,['rcv_bc_code']] = np.ravel(iban_cc.loc[iban_cc['Bank'] == bc_bank,['Code']])
    timestamps = datetime.strptime(timestamps,"%Y.%m.%d %H:%M")
    trx_data.loc[i,['valuta_date']] = valuta = timestamps + timedelta(hours=12*i)
    trx_data.loc[i,['cust_vertrag_nr']] = cd_contract = np.random.choice(np.ravel(cust_data['vertrag_nr']))
    trx_data.loc[i,['cust_konto_nr']] = np.ravel(cust_data.loc[cust_data['vertrag_nr'] == cd_contract,['konto_nr']])
    trx_data.loc[i,['cust_vname']] = np.ravel(cust_data.loc[cust_data['vertrag_nr'] == cd_contract,['vorname']])
    trx_data.loc[i,['cust_nname']] = np.ravel(cust_data.loc[cust_data['vertrag_nr'] == cd_contract,['nachname']])
    trx_data.loc[i,['cust_plz']] = np.ravel(cust_data.loc[cust_data['vertrag_nr'] == cd_contract,['plz']])
    trx_data.loc[i,['cust_ort']] = np.ravel(cust_data.loc[cust_data['vertrag_nr'] == cd_contract,['ort']])
    trx_data.loc[i,['cust_kanton']] = np.ravel(cust_data.loc[cust_data['vertrag_nr'] == cd_contract,['kanton']])
    trx_data.loc[i,['cust_sprachregion']] = np.ravel(cust_data.loc[cust_data['vertrag_nr'] == cd_contract,['sprachregion']])
    trx_data.loc[i,['cust_auth_device']] = np.ravel(cust_data.loc[cust_data['vertrag_nr'] == cd_contract,['auth_device']])
    
trx_data.head(30)




,timestamp,paym_id,send_bc_nr,amount,rcv_bc_nr,rcv_bc_iban,rcv_bc_country,rcv_bc_code,valuta_date,cust_vertrag_nr,cust_konto_nr,cust_vname,cust_nname,cust_plz,cust_ort,cust_kanton,cust_sprachregion,cust_auth_device
0,2018.01.01 08:00,1751202176206,CHBANK71XXX,27972,TRBANK78XXX,TR020001000201529153355002,Türkei,TR,2018-01-01 08:00:00,65094,CH5300772636881565787,Thomas,Zingg,6260,Reiden,LU,de,mtan
1,2018.01.01 08:00,1384509894315,CHBANK71XXX,75144,GEBANK29XXX,GE02TB7523045063700002,Georgia,GE,2018-01-01 20:00:00,65300,CH5300772951191211846,Luca,Zurfluh,6170,Schüpfheim,LU,de,ptan
2,2018.01.01 08:01,1875373084962,CHBANK71XXX,97827,SMBANK67XXX,SM07U0854009803000030174419,San Marino,SM,2018-01-02 08:01:00,79380,CH5300772993220232113,Paola,Dazio,6805,Mezzovico,TI,it,mtan
3,2018.01.01 08:01,1138979463689,CHBANK71XXX,68423,PSBANK62XXX,PS06ARAB000000009040781605610,Palästina,PS,2018-01-02 20:01:00,82743,CH5300772051301868157,Maria,Etter,6287,Aesch LU,LU,de,mtan
4,2018.01.01 08:02,1040125959833,CHBANK71XXX,19795,BEBANK15XXX,BE45096920886089,Belgien,BE,2018-01-03 08:02:00,75442,CH5300772121900454541,Claudia,Maurizio,6288,Schongau,LU,de,ptan
5,2018.01.01 08:02,1890525479883,CHBANK71XXX,57304,ALBANK11XXX,AL06202111090000000005012075,Albanien,AL,2018-01-03 20:02:00,70633,CH5300772950673929586,Monika,Brun,6044,Udligenswil,LU,de,ptan
6,2018.01.01 08:03,1781064360424,CHBANK71XXX,75171,BRBANK17XXX,BR0200000000010670000117668C1,Brasilien,BR,2018-01-04 08:03:00,68923,CH5300772615207750577,Salvatore,Bettazza,6204,Sempach,LU,de,ptan
7,2018.01.01 08:03,1985231384668,CHBANK71XXX,5932,MKBANK54XXX,MK07200000625758632,Mazedonien,MK,2018-01-04 20:03:00,80639,CH5300772190943206794,Michael,Kern,9064,Hundwil,AR,de,mtan
8,2018.01.01 08:04,1639723910653,CHBANK71XXX,3078,MCBANK56XXX,MC2412739000710075018000P14,Monaco,MC,2018-01-05 08:04:00,84019,CH5300772741174721416,Antonio,Ferreira,3178,Bösingen,FR,de,ptan
9,2018.01.01 08:04,1426960606339,CHBANK71XXX,95917,ITBANK39XXX,IT43K0310412701000000820420,Italien,IT,2018-01-05 20:04:00,81826,CH5300772217819228016,Andrea,Schirmer,9410,Heiden,AR,de,mtan


In [149]:


import pandas as pd
from random import randint

df = pd.DataFrame(columns=['A','B','C','D','E'])

for i in range(5): #add 5 rows of data
    df.loc[i, ['A']] = randint(0,99)
    df.loc[i, ['B']] = randint(0,99)
    df.loc[i, ['C']] = randint(0,99)
    df.loc[i, ['D']] = randint(0,99)
    df.loc[i, ['E']] = randint(0,99)
    
df

,A,B,C,D,E
0,82.0,93.0,1.0,5.0,8.0
1,4.0,34.0,45.0,13.0,88.0
2,87.0,41.0,54.0,41.0,43.0
3,81.0,44.0,98.0,3.0,83.0
4,47.0,52.0,20.0,45.0,60.0
